In [46]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [10]:
import zarr
from candidate_graph import create_candidate_graph
import daisy
import linajea

In [7]:
candidate_graph = create_candidate_graph('anno_alice_T2030_tiny.zarr')

/
 ├── Fragment_stats
 │   ├── Position
 │   ├── Volume
 │   └── id
 ├── Fragments (3, 15, 30, 30) uint64
 ├── Merge_tree
 │   ├── Merge
 │   └── Scoring
 ├── Raw (4, 3, 15, 30, 30) float64
 └── gt_trackimage (3, 15, 30, 30) uint64
add 35321834118144 985274108615169 count 0.03571428571428571
 iterating time:  0.03549361228942871
add 35321834118144 1196380341148161 count 0.029411764705882353
 iterating time:  0.03796863555908203
add 35321834118144 27725491381213185 count 0.0012690355329949238
 iterating time:  0.06509518623352051
add 35321834118144 462041294558989825 count 7.614986293024673e-05
 iterating time:  0.8224177360534668
add 35321834118144 462920903861210625 count 7.60051683514479e-05
 iterating time:  0.8962619304656982
add 35321834118144 470696650092841985 count 7.474958887726118e-05
 iterating time:  0.9706876277923584
add 35321834118144 472702159301905921 count 7.44324525493115e-05
 iterating time:  1.0469982624053955
add 35321834118144 473229924883238401 count 7.434944237

In [55]:
G = candidate_graph

In [56]:
import networkx as nx
for node in G.nodes:
    if  G.nodes[node]['parent'] is None:
        G.nodes[node]['parent'] = node

In [57]:
roi = daisy.Roi((0, 0, 0, 0), (3, 15, 25, 25))

    # input graph
graph = linajea.tracking.TrackGraph(G, frame_key='t', roi=roi)

In [58]:
from linajea_cost_test import get_edge_indicator_costs, get_node_indicator_costs, get_constraints


import time
import logging

logger = logging.getLogger(__name__)


class TrackingConfig():
    def __init__(self, solve_config):
        self.solve = solve_config

In [59]:
ps = {
            "track_cost": 4.0,
            "weight_edge_score": -0.1,
            "weight_node_score": -0.1,
            "selection_constant": -1.0,
            "max_cell_move": 0.0,
            "block_size": [5, 100, 100, 100],
            }
job = {"num_workers": 5, "queue": "normal"}

solve_config = linajea.config.SolveConfig(parameters=ps,
                                            job=job,
                                            context=[2, 100, 100, 100])

solve_config.solver_type = None
solve_config.timeout = 100000

In [60]:
config = TrackingConfig(solve_config)
constrs = get_constraints()

linajea.tracking.track(
                graph,
                config,
                frame_key='t',
                selected_key='selected',
                edge_indicator_costs=get_edge_indicator_costs,
                node_indicator_costs=get_node_indicator_costs,
                constraints_fns=constrs[0],
                pin_constraints_fns=constrs[1]
                )


Could not create Gurobi backend: Gurobi error in pylp/impl/solvers/GurobiBackend.cpp:22: No Gurobi license found (user silja, host Precision-5820-Tower, hostid e5051bc3, cores 12)
feasible solution found by trivial heuristic after 0.1 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       363 del vars, 737 del conss, 0 add conss, 187 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 589 clqs
(round 2, exhaustive) 363 del vars, 737 del conss, 0 add conss, 187 chg bounds, 0 chg sides, 0 chg coeffs, 17430 upgd conss, 0 impls, 616 clqs
(round 3, medium)     698 del vars, 16935 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 334 chg coeffs, 17430 upgd conss, 0 impls, 32982 clqs
(round 4, exhaustive) 698 del vars, 33132 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 655 chg coeffs, 17430 upgd conss, 0 impls, 32982 clqs
(round 5, medium)     791 del vars, 33225 del conss, 32394 add conss, 187 chg bounds, 334 chg sides, 655 chg coeffs, 17430 